# Running MegaDetector on camera trap images using Google Colab

<a href="https://colab.research.google.com/github/microsoft/CameraTraps/blob/master/detection/megadetector_colab.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a>

Link in case the above badge doesn't redirect you correctly: [Open in Colab](https://colab.research.google.com/github/microsoft/CameraTraps/blob/master/detection/megadetector_colab.ipynb)

Adapted from older versions by [@louis030195](https://github.com/louis030195) 
and [@alsnothome](https://github.com/alsnothome).

For reference please read the [MegaDetector guide on GitHub](https://github.com/microsoft/CameraTraps/blob/master/megadetector.md) and check there for updates. Here we have roughly followed the steps for running under Linux.

This notebook is designed to load camera trap image files already uploaded onto Google Drive. If you don't have images already loaded onto Google Drive or just want to see a demo of MegaDetector in action, we also provide code to download some sample images.

The steps walk through copying of all of the required model and helper files to the Colab runtime and installing all the required packages. You can then connect to your Google Drive folder and process all of the images in a folder using the MegaDetector saved model. The output is saved in a JSON file - a text based database file whose format is described in this [section](https://github.com/microsoft/CameraTraps/tree/master/api/batch_processing#batch-processing-api-output-format) in the batch API user guide. The detections (as bounding boxes) can then be rendered on your images.

The Google Colab instance will only stay open for a maximum 10-12 hrs and after that it will close and any unsaved data will be lost. We recommend saving the JSON output and annotated images into your Google Drive folder for persistent storage.

## Set up the Colab instance to run on GPU processing


Navigate to Edit→Notebook Settings and select "GPU" from the Hardware Accelerator drop-down 

## Install dependencies, download the model, set PYTHONPATH

Note: from here on you'll start seeing a mix of code. Most are Linux system commands, rather than Python. The system commands are prefixed by a shebang `!`, which tells this notebook to execute them on the command line.

### Install required Python packages

In [ ]:
pip install humanfriendly jsonpickle

### Import TensorFlow for TFODAPI v1.0 models (using the tf.compat.v1 module)

In [ ]:
import tensorflow.compat.v1 as tf
print(tf.__version__)

### Download the MegaDetector model file

MegaDetector v4.1, 2020.04.27, frozen model (.pb)

In [ ]:
!wget -O /content/megadetector_v4_1_0.pb https://lilablobssc.blob.core.windows.net/models/camera_traps/megadetector/md_v4.1.0/md_v4.1.0.pb

### Clone the two required Microsoft git repos
This will copy the latest version of the Microsoft AI for Earth "utilities" and "Camera Traps" repositories from GitHub. These make data handling and running the model easy. 

In [ ]:
!git clone https://github.com/microsoft/CameraTraps
!git clone https://github.com/microsoft/ai4eutils

### Set `PYTHONPATH` to include `CameraTraps` and `ai4eutils`

Add cloned git folders to the `PYTHONPATH` environment variable so that we can import their modules from any working directory.


In [ ]:
import os
os.environ['PYTHONPATH'] += ":/content/ai4eutils"
os.environ['PYTHONPATH'] += ":/content/CameraTraps"

!echo "PYTHONPATH: $PYTHONPATH"

## Mount Google Drive in Colab
You can mount your Google Drive if you have sample images there to try MegaDetector on or want to save the results to your Google Drive.

Once you run the cell below, it will show a URL and a text box.

Visit that URL to choose the Google account where the images you want to process live. After you authenticate, an authorization code will be shown. Copy the authorization code to the text box here. 

Your Google Drive folders will then be mounted under `/content/drive` and can be viewed and navigated in the Files pane.

The method is described under this Colab code snippet: https://colab.research.google.com/notebooks/io.ipynb#scrollTo=u22w3BFiOveA. Never give out your account username and password. Read this Colab code snippet to understand how this connection is made and authenticated. There are other ways to connect your Google Drive or upload your data if you do not find this method suitable.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Download sample images

We install Microsoft Azure's [azcopy](https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-v10) utility, which we then use to download camera trap images from the [Snapshot Serengeti](http://lila.science/datasets/snapshot-serengeti) dataset hosted on [lila.science](http://lila.science).

In [ ]:
%%bash

# download azcopy
wget -O azcopy_linux.tar.gz https://aka.ms/downloadazcopy-v10-linux
tar -xvzf azcopy_linux.tar.gz --wildcards */azcopy --strip 1
rm azcopy_linux.tar.gz
chmod u+x azcopy

# copy Snapshot Serengeti images to a local directory
DATASET_URL="https://lilablobssc.blob.core.windows.net/snapshotserengeti-unzipped/"
SAMPLE_DIR="S1/D05/D05_R4"
LOCAL_DIR="/content/snapshotserengeti"

./azcopy cp "${DATASET_URL}${SAMPLE_DIR}" "${LOCAL_DIR}" --recursive

In [ ]:
# this should show 48 images downloaded
!ls /content/snapshotserengeti/ -R

## MegaDetector batch processing

This step executes the Python script `run_tf_detector_batch.py` that we copied from the CameraTraps repo. It has three mandatory arguments and one optional:

1.   path to the MegaDetector saved model file.
2.   a folder containing images. If your images were already on Google Drive, replace `[Image_Folder]` with your folder name from Google Drive. If you are using the sample images from Snapshot Serengeti, change `images_dir` to `'/content/snapshotserengeti'`.
3.   the output JSON file location and name - replace `[Output_Folder]` with your folder name and `[output_file_name.json]` with your file name.
4.   option `--recursive` goes through all subfolders to find and process all images within.

You will need to change the image folder path and output file path, depending on your situation.

In our experience the Colab system will take ~30 seconds to initialize and load the saved MegaDetector model. It will then iterate through all of the images in the folder specified. Processing initially takes a few seconds per image and usually settles to ~1 sec per image. That is ~60 images per minute or ~3600 images per hour. Limit the number of images in your folder so that all of the processing can be completed before the Colab session ends.

If you see the error "AssertionError: output_file specified needs to end with .json" then you haven't update the output folder and file name in the line of code below properly.

In [ ]:
images_dir = '/content/snapshotserengeti'

# choose a location for the output JSON file
output_file_path = '/content/drive/My Drive/snapshotserengeti-test/snapshot-serengeti-mdv4.json'

Here we pass the Python variable value `output_file_path` you specified above to the bash commands below using `$` (double quoting as there are spaces in this path), to run the script. This is so that we can refer to the output file path later for visualization.

In [ ]:
!ls /content/CameraTraps/detection

In [ ]:
!python /content/CameraTraps/detection/run_detector_batch.py megadetector_v4_1_0.pb "$images_dir" "$output_file_path" --recursive

## Visualize batch processing script outputs

Here we use the `visualize_detector_output.py` in the `visualization` folder of the Camera Traps repo to see the output of the MegaDetector visualized on our images. It will save images annotated with the results (original images will *not* be modified) to the folder you specify here.

The scripts take in a number of optional parameters to control output image size and how many are sampled (if you've processed a lot of images but only want to visualize the results on a few) - take a look at the `main()` function in the script to see what other parameters are available.

In [ ]:
visualization_dir = '/content/visualized_images'  # pick a location for annotated images

In [ ]:
!python /content/CameraTraps/visualization/visualize_detector_output.py "$output_file_path" "$visualization_dir" --confidence 0.8 --images_dir "$images_dir"

In [ ]:
import os
from PIL import Image

In [ ]:
for viz_file_name in os.listdir(visualization_dir):
  print(viz_file_name)
  im = Image.open(os.path.join(visualization_dir, viz_file_name))
  display(im)

# Next steps

Now that you have tried applying the MegaDetector on your own images and assessed its effectiveness, here are some pointers to help you take advantage of the MegaDetector to label your survey images more quickly.

### Ways to use the output JSON in a camera trap image processing workflow 

#### 1. Timelapse

[Timelapse](http://saul.cpsc.ucalgary.ca/timelapse/pmwiki.php?n=Main.HomePage) is an open-source tool for annotating camera trap images. We have worked with the Timelapse developer to integrate the output of our API into Timelapse, so a user can:

- Select or sort images based on whether they contain animal or people or vehicles.
- View bounding boxes during additional manual annotation steps (which may speed up review...)

See our [Integration with Timelapse](https://github.com/microsoft/CameraTraps/blob/master/api/batch_processing/integration/timelapse.md) page for more information.

![Screenshot showing the Timelapse application with MegaDetector output, shown as a bounding box around the detected animal](https://github.com/microsoft/CameraTraps/blob/master/api/batch_processing/integration/images/tl_boxes.jpg?raw=1)


**Subsetting the output JSON into more manageable JSONs to use with Timelapse**

People using Timelapse often split up the workload and have multiple Timelapse databases per study. We wrote a simple Windows desktop app ([Camera Trap Batch API Output Manager App](https://github.com/microsoft/CameraTraps/blob/master/api/batch_processing/postprocessing/CameraTrapJsonManagerApp.md)) for splitting up the output JSON.


#### 2. Separating images into folders that contain animals/people/vehicles/nothing

Some of our collaborators do their downstream labeling without Timelapse, by moving the images to separate folders containing animals/people/vehicles/nothing according to MegaDetector output. You can use the script [separate_detections_into_folders.py](https://github.com/microsoft/CameraTraps/blob/master/api/batch_processing/postprocessing/separate_detections_into_folders.py) to do that if you have some familiarity with running Python scripts. This script has only one external package ("tqdm") to install - you can also follow our [installation instructions](https://github.com/microsoft/CameraTraps#initial-setup) to run the script inside a conda virtual environment. 


### Help me process millions and millions of images

We often run MegaDetector on behalf of users, email us at [cameratraps@lila.science](mailto:cameratraps@lila.science) if you need help running large batches.